In [ ]:
import random
import networkx as nx
import numpy as np
from community import community_louvain
import matplotlib.pyplot as plt
import collections
from collections import Counter

In [ ]:
import pandas as pd
import networkx as nx
_DEBUG_ = False
def load_data(file_name):
    """
    read edges from an edge file
    """
    edges = list()
    df = pd.read_csv(file_name)
    for idx, row in df.iterrows():
        user_id, friends = row["user_id"], eval(row["friends"])
        for friend in friends:
            # add each friend relation as an edge
            edges.append((user_id, friend))
    edges = sorted(edges)
    
    return edges
def buildGraph(G,code_edge):
    for edge in code_edge:
        G.add_edge(int(edge[0]),int(edge[1]),weight=1.0)

# This method keeps removing edges from Graph until one of the connected components of Graph splits into two
# compute the edge betweenness
def CmtyGirvanNewmanStep(G):
    #global ids
    #print(ids)
    if _DEBUG_:
        print("Running CmtyGirvanNewmanStep method ...")
    init_ncomp = nx.number_connected_components(G)    #no of components
    ncomp = init_ncomp
    while ncomp <= init_ncomp:
        bw = nx.edge_betweenness_centrality(G, weight='weight')    #edge betweenness for G
        #find the edge with max centrality
        max_ = max(bw.values())
        #find the edge with the highest centrality and remove all of them if there is more than one!
        for k, v in bw.items():
            if float(v) == max_:
                G.remove_edge(k[0],k[1])    #remove the central edge
                #ids+=1
                #print("Process "+str(ids))
        ncomp = nx.number_connected_components(G)    #recalculate the no of components


# This method compute the modularity of current split
def _GirvanNewmanGetModularity(G, deg_, m_):
    New_A = nx.adj_matrix(G)
    New_deg = {}
    New_deg = UpdateDeg(New_A, G.nodes())
    #Let's compute the Q
    comps = nx.connected_components(G)    #list of components    
    print('No of communities in decomposed G: {}'.format(nx.number_connected_components(G)))
    Mod = 0    #Modularity of a given partitionning
    for c in comps:
        EWC = 0    #no of edges within a community
        RE = 0    #no of random edges
        for u in c:
            EWC += New_deg[u]
            RE += deg_[u]        #count the probability of a random edge
        Mod += ( float(EWC) - float(RE*RE)/float(2*m_) )
    Mod = Mod/float(2*m_)
    if _DEBUG_:
        print("Modularity: {}".format(Mod))
    return Mod


def UpdateDeg(A, nodes):
    deg_dict = {}
    n = len(nodes)  #len(A) ---> some ppl get issues when trying len() on sparse matrixes!
    B = A.sum(axis = 1)
    i = 0
    for node_id in list(nodes):
        deg_dict[node_id] = B[i, 0]
        i += 1
    return deg_dict


# This method runs GirvanNewman algorithm and find the best community split by maximizing modularity measure
def runGirvanNewman(G, Orig_deg, m_):
    #let's find the best split of the graph
    BestQ = 0.0
    Q = 0.0
    while True:    
        CmtyGirvanNewmanStep(G)
        Q = _GirvanNewmanGetModularity(G, Orig_deg, m_);
        print("Modularity of decomposed G: {}".format(Q))
        if Q > BestQ:
            BestQ = Q
            Bestcomps = list(nx.connected_components(G))    #Best Split
            print("Identified components: {}".format(Bestcomps))
        if G.number_of_edges() == 0:
            break
    if BestQ > 0.0:
        print("Max modularity found (Q): {} and number of communities: {}".format(BestQ, len(Bestcomps)))
        print("Graph communities: {}".format(Bestcomps))
    else:
        print("Max modularity (Q):", BestQ)

train_edge = load_data('data/train.csv')
count=0
nodeToInt = {}
code_edge = list()
for edge in train_edge:
    node1 = edge[0]
    node2 = edge[1]
    if node1 not in nodeToInt.keys():
        nodeToInt[node1] =count
        count+=1
    if node2 not in nodeToInt.keys():
        nodeToInt[node2] = count
        count+=1
    code_edge.append((nodeToInt[node1],nodeToInt[node2]))
G = nx.Graph()
buildGraph(G,code_edge)
n = G.number_of_nodes()    #|V|
A = nx.adj_matrix(G)    #adjacenct matrix

m_ = 0.0    #the weighted version for number of edges
m = len(code_edge)
#m_ = m_/2.0
if _DEBUG_:
    print("m: {}".format(m_))



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:115: DeprecationWarning: adj_matrix is deprecated and will be removed in version 3.0.
Use `adjacency_matrix` instead



In [ ]:
comms = community_louvain.best_partition(G)
unique_coms = np.unique(list(comms.values()))


In [ ]:
import torch

In [ ]:
comms.values()

dict_values([0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 0, 0, 2, 2, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 3, 3, 4, 4, 1, 1, 1, 7, 7, 0, 0, 0, 0, 8, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2, 2, 3, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 3, 3, 3, 9, 9, 3, 2, 2, 2, 3, 3, 0, 1, 1, 1, 1, 1, 2, 8, 8, 3, 3, 3, 3, 0, 10, 10, 10, 10, 1, 1, 3, 1, 2, 3, 3, 0, 0, 11, 11, 12, 12, 12, 12, 1, 12, 12, 12, 12, 3, 12, 12, 12, 12, 12, 1, 12, 12, 12, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 37, 37, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 

In [ ]:
def visualize(h, color, epoch=None, loss=None, accuracy=None):
    plt.figure(figsize=(100,100))
    plt.xticks([])
    plt.yticks([])
 
    if torch.is_tensor(h):
        h = h.detach().cpu().numpy()
        plt.scatter(h[:, 0], h[:, 1], s=10, c=color, cmap="Set2")
        if epoch is not None and loss is not None and accuracy['train'] is not None and accuracy['val'] is not None:
            plt.xlabel((f'Epoch: {epoch}, Loss: {loss.item():.4f} \n'
                       f'Training Accuracy: {accuracy["train"]*100:.2f}% \n'
                       f' Validation Accuracy: {accuracy["val"]*100:.2f}%'),
                       fontsize=16)
    else:
        nx.draw_networkx(h, pos=nx.spring_layout(h, seed=42), with_labels=False,
                         node_color=color, cmap="Set2")
    plt.show()


In [ ]:
import collections

In [ ]:
from collections import defaultdict

In [ ]:
def construct_graph_from_edges(edges):
    """
    generate a directed graph object given true edges
    DiGraph documentation: https://networkx.github.io/documentation/stable/reference/classes/digraph.html
    """
    # convert a list of edges {(u, v)} to a list of edges with weights {(u, v, w)}
    edge_weight = defaultdict(float)
    for e in edges:
        edge_weight[e] += 1.0
    weighed_edge_list = list()
    for e in sorted(edge_weight.keys()):
        weighed_edge_list.append((e[0], e[1], edge_weight[e]))
        
    graph = nx.DiGraph()
    graph.add_weighted_edges_from(weighed_edge_list)
    
    print("number of nodes:", graph.number_of_nodes())
    print("number of edges:", graph.number_of_edges())
    
    return graph

In [ ]:
s  = np.array(list(comms.values()))

In [ ]:
GG = construct_graph_from_edges(train_edge)

number of nodes: 8343
number of edges: 100000


In [ ]:
nx.number_connected_components(GG.to_undirected())

192

In [ ]:
comms = community_louvain.best_partition(GG.to_undirected())
unique_coms = np.array(list(comms.values()))

In [ ]:
value = torch.from_numpy(np.array(list(comms.values())))

In [ ]:
visualize(GG,value)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
nodes = list(GG.nodes())

In [ ]:
degree = dict(GG.degree)

In [ ]:
for node in nodes:
  if degree[node]<10:
    GG.remove_node(node)

In [ ]:
GG.number_of_edges()

84063

In [ ]:
comms = community_louvain.best_partition(GG.to_undirected())
unique_coms = np.array(list(comms.values()))

In [ ]:
value = torch.from_numpy(np.array(list(comms.values())))

In [ ]:
visualize(GG,value)

Output hidden; open in https://colab.research.google.com to view.